
<h1 align="center"> Segmentation and Clustering - Toronto </h1>


<h3>Q1: Scraping</h3>

In [5]:
# Fetch webpage
import requests
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url_response = requests.get(url)
url_response

<Response [200]>

In [20]:
#Extracting Data
soup=BeautifulSoup(url_response.text,'lxml')
soup.title

postalCodeList = []
boroughList = []
neighborhoodList = []
for row in soup.table.find("tbody").find_all("tr"):
    cells = row.find_all("td")
    for cell in cells:
        i = cell.get_text(separator='*', strip=True).split('*')
        a_data = cell.find_all("a")
        nei = ''
        if len(a_data)>1:
            for a in a_data[1:]:
                if nei == '': 
                    nei += a_data[1].text
                else:
                    nei = nei + ", " + a.text
        if len(a_data)>1:
            postalCodeList.append(i[0])
            boroughList.append(a_data[0].text)
            neighborhoodList.append(nei)

In [25]:
# Transfer data to dataframe
import pandas as pd
toronto_df = pd.DataFrame({"Postal Code": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M9A,Etobicoke,Islington Avenue


In [26]:
# Groupin data
toronto_group = toronto_df.groupby(['Postal Code', 'Borough'], as_index=False).agg(lambda x: ", ".join(x))
toronto_group.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [27]:
# Shape of Data
toronto_group.shape

(87, 3)

In [29]:
# Exporting Data
toronto_group.to_csv('toronto.csv', index=False)

<br>

<h3>Q2: Adding Latitude and Longitude Co-ordinates to the DataFrame</h3>

In [30]:
!pip install geocoder

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 12.3 MB/s eta 0:00:01


In [31]:
import numpy as np
import geocoder

In [32]:
toronto_group.describe()

,Postal Code,Borough,Neighborhood
count,87,87,87
unique,87,17,83
top,M6J,North York,Downsview
freq,1,23,3


In [33]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

[43.709020000000066, -79.36348999999996]

In [35]:
# Fetch Postal Code Co-ordinates
postal_codes = toronto_group['Postal Code']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
toronto_group['Latitude'] = df_coords['Latitude']
toronto_group['Longitude'] = df_coords['Longitude']

In [37]:
toronto_group.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892


In [38]:
# Export data
toronto_group.to_csv('toronto_with_coordinates.csv',index=False)

<br>

<h3>Q3: Segmenting and Clustering Neighborhoods</h3>

In [39]:
# Install
!pip install geopy
!pip install folium

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 8.3 MB/s  eta 0:00:01


In [40]:
# Import
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

In [41]:
toronto_group.shape

(87, 5)

In [46]:
#Use geopy library to get the latitude and longitude values of Toronto
address = 'Toronto'
geolocator = Nominatim(user_agent="http")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<h4>Map of Toronto with neighborhoods superimposed on top</h4>

In [48]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_group['Latitude'], toronto_group['Longitude'], toronto_group['Borough'], toronto_group['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

<h4>Filter only boroughs that contain the word Toronto</h4>

In [49]:
# filter borough names that contain the word Toronto
borough_names = list(toronto_group.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['Downtown Toronto']

In [55]:
toronto_data = toronto_group[toronto_group['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.shape

(15, 5)

In [56]:
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.68190,-79.37829
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.66788,-79.36649
2,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804


In [57]:
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7,parse_html=False).add_to(map_toronto)  
map_toronto

In [58]:
# Foursquare API
CLIENT_ID = 'QVAKSVGCCBRWGHL1RNAFKM0WN2K23ZXIHYL5ZGG3TSYF2C5G'
CLIENT_SECRET = '0CJV3400WNUVMLG10LGQRC5WX1NA1FR2N11KSEXYTDZJ5E5X'
VERSION = '20180604'
LIMIT = 30

<h4>Exploring Neighbourhood in Toronto</h4>

In [59]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([( name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [60]:
df = toronto_data
toronto_venues = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'],longitudes=df['Longitude'])

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Bay Street
Richmond, King
Harbourfront, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport
First Canadian Place, Underground city


In [61]:
toronto_venues.shape

(415, 7)

In [62]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.68190,-79.37829,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.68190,-79.37829,Park Drive Reservation Lands,43.679822,-79.377787,Park
2,Rosedale,43.68190,-79.37829,Betline Trail at Roxborough dr.,43.680530,-79.381490,Bike Trail
3,Rosedale,43.68190,-79.37829,Whitney Park,43.682036,-79.373788,Park
4,"St. James Town, Cabbagetown",43.66788,-79.36649,Cranberries,43.667843,-79.369407,Diner


In [63]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bay Street,30,30,30,30,30,30
"CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport",30,30,30,30,30,30
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
"First Canadian Place, Underground city",30,30,30,30,30,30
"Garden District, Ryerson",30,30,30,30,30,30
"Harbourfront, Union Station, Toronto Islands",30,30,30,30,30,30
"Kensington Market, Chinatown, Grange Park",30,30,30,30,30,30
"Regent Park, Harbourfront",21,21,21,21,21,21


In [64]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 123 uniques categories.


<h4>Analyze Each Borough Neighborhood</h4>

In [65]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,...,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:

toronto_onehot.shape

(415, 123)

In [67]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,...,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Bay Street,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Commerce Court, Victoria Hotel",0.000000,0.066667,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"First Canadian Place, Underground city",0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Harbourfront, Union Station, Toronto Islands",0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
7,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.033333,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.033333
8,"Regent Park, Harbourfront",0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,...,0.000000,0.000000,0.047619,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Richmond, King",0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000


In [68]:
toronto_grouped.shape

(15, 123)

In [69]:

num_top_venues = 5
for neigh in toronto_grouped['Neighborhood']:
    print("----"+neigh+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bay Street----
                venue  freq
0         Coffee Shop  0.10
1      Clothing Store  0.07
2  Italian Restaurant  0.03
3   Electronics Store  0.03
4       Shopping Mall  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport----
                  venue  freq
0    Italian Restaurant  0.10
1  Gym / Fitness Center  0.07
2            Restaurant  0.07
3                  Park  0.07
4           Yoga Studio  0.03


----Church and Wellesley----
             venue  freq
0      Coffee Shop  0.10
1      Men's Store  0.07
2          Gay Bar  0.07
3        Juice Bar  0.03
4  Bubble Tea Shop  0.03


----Commerce Court, Victoria Hotel----
                 venue  freq
0                 Café  0.13
1  Japanese Restaurant  0.07
2            Gastropub  0.07
3          Coffee Shop  0.07
4  American Restaurant  0.07


----First Canadian Place, Underground city----
                 venue  freq
0                 Café  0.20
1           Restaurant  0.10
2  J

In [70]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [71]:

import numpy as np
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(15, 11)

<h4>Clustering Neighborhoods</h4>

In [73]:
from sklearn.cluster import KMeans
from sklearn.cluster import k_means_
km = KMeans(n_clusters=3, init='k-means++', max_iter=100, n_init=1, 
  verbose=True)

In [74]:
kclusters = 10
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)
print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

[1 3 8 0 0 6 1 9 5 0]
15


In [75]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.68190,-79.37829
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.66788,-79.36649
2,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804


In [76]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.68190,-79.37829,2,Park,Bike Trail,Playground,Wine Bar,Event Space,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.66788,-79.36649,7,Café,Restaurant,Italian Restaurant,Coffee Shop,Park,Diner,Pub,Playground,Gift Shop,Deli / Bodega
2,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133,8,Coffee Shop,Gay Bar,Men's Store,Pub,Burger Joint,Sake Bar,Diner,Restaurant,Juice Bar,Ramen Restaurant
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,5,Coffee Shop,Breakfast Spot,Yoga Studio,Distribution Center,Electronics Store,Event Space,Restaurant,Spa,Food Truck,Pub
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,6,Café,Theater,Ramen Restaurant,Clothing Store,Hotel,Plaza,Diner,Burger Joint,Burrito Place,Lounge


In [77]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters